In [1]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [5]:
test['id'].dtype

dtype('int64')

In [7]:
test.id.values.dtype

dtype('int64')

In [11]:
# 我们用不上 id
idx = test.id.values.astype(int)
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

In [10]:
idx.dtype

dtype('int64')

In [13]:
train

,query,product_title,product_description,median_relevance,relevance_variance
0,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000
1,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000
2,projector,ViewSonic Pro8200 DLP Multimedia Projector,NaN,4,0.471
3,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",4,0.000
4,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",2,0.471
5,oakley polarized radar,Oakley Sunglasses - Radar Path Polished Black/...,"Durability and all-day comfort of lightweight,...",3,1.166
6,boyfriend jeans,How To Make An American Quilt (DVD),ITEM#: 2518897\nDirector Jocelyn Moorhouse's f...,1,0.000
7,screen protector samsung,ZAGG InvisibleShield Cell Phone Screen Protect...,Protect your most precious technology with the...,4,0.000
8,pots and pans set,Cook N Home Stainless Steel 4-Piece Pasta Cook...,This ultimate pasta and vegetable steamer cook...,2,0.632
9,waffle maker,Presto FlipSide Electric Waffle Maker- 03510,Please the waffle lover in your family with th...,4,0.471


把 query 和 product_title 拼接在一起。

In [14]:
# do some lambda magic on text columns
traindata = list(train.apply(lambda x: '%s %s' %
                             (x['query'], x['product_title']), axis=1))
testdata = list(test.apply(lambda x: '%s %s' %
                           (x['query'], x['product_title']), axis=1))

In [18]:
traindata[0]

'bridal shower decorations Accent Pillow with Heart Design - Red/Black'

In [19]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition, pipeline, metrics, grid_search

# the infamous tfidf vectorizer (Do you remember this one?)
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
        strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
        ngram_range=(1, 5), use_idf=1,smooth_idf=1,sublinear_tf=1,
        stop_words = 'english')

/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [20]:
# Fit TFIDF
tfv.fit(traindata)
X = tfv.transform(traindata)
X_test = tfv.transform(testdata)

In [ ]:
# Initialize SVD
svd = TruncatedSVD()

# Initialize the standard scaler
scl = StandardScaler()

# We will use SVM here..
svm_model = SVC()

# Create the pipeline
clf = pipeline.Pipeline([('svd', svd),
                         ('scl', scl),
                         ('svm', svm_model)])

# Create a parameter grid to search for best parameters for everything in the pipeline
param_grid = {'svd__n_components': [200, 400],
              'svm__C': [10, 12]}

In [ ]:

    # Kappa Scorer 
    kappa_scorer = metrics.make_scorer(quadratic_weighted_kappa, greater_is_better = True)
    
    # Initialize Grid Search Model
    model = grid_search.GridSearchCV(estimator = clf, param_grid=param_grid, scoring=kappa_scorer,
                                     verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)
                                     
    # Fit Grid Search Model
    model.fit(X, y)
    print("Best score: %0.3f" % model.best_score_)
    print("Best parameters set:")
    best_parameters = model.best_estimator_.get_params()
    for param_name in sorted(param_grid.keys()):
    	print("\t%s: %r" % (param_name, best_parameters[param_name]))
    
    # Get best model
    best_model = model.best_estimator_
    
    # Fit model with best parameters optimized for quadratic_weighted_kappa
    best_model.fit(X,y)
    preds = best_model.predict(X_test)
    
    # Create your first submission file
    submission = pd.DataFrame({"id": idx, "prediction": preds})
    submission.to_csv("beating_the_benchmark_yet_again.csv", index=False)